#### 輸入起始點、終點座標、法向量，生成掃除路徑

In [24]:
# 函數作用: 通過兩點生成線段
from OCC.Core.gp import gp_Pnt
from OCC.Core.GC import GC_MakeSegment
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir,gp_Circ, gp_Ax2, gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire,BRepBuilderAPI_MakeFace
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.Display.SimpleGui import init_display
import numpy as np

def grinding_area(point_start,point_end,normal_vector):
    #根據起始、終點生成線段
    point_s=gp_Pnt(float(point_start[0]),float(point_start[1]), float(point_start[2]))
    point_e=gp_Pnt(float(point_end[0]),float(point_end[1]), float(point_end[2]))
    aSegment = GC_MakeSegment(point_s, point_e )
    anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
    aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())

    #input
    #normal_vector
    magnitude = np.linalg.norm(normal_vector)# 將法向量除以其範數以得到單位法向量
    normal_vector = normal_vector/ magnitude

    #two point_position
    tan_vector=point_start-point_end

    #direction_1
    dir_1= np.cross(normal_vector, tan_vector)
    magnitude = np.linalg.norm(dir_1)# 將向量除以其範數以得到單位向量
    dir_1 = dir_1/ magnitude

    # 定義截面為方形
    # 定義方形的四個頂點
    depth=0.01#每次研磨深度
    width=10#研磨寬度
    point_1=point_start+normal_vector*depth+dir_1*width
    point_2=point_start+normal_vector*depth-dir_1*width
    point_3=point_start-normal_vector*50-dir_1*width
    point_4=point_start-normal_vector*50+dir_1*width
    p1 = gp_Pnt(point_1[0], point_1[1], point_1[2])
    p2 = gp_Pnt(point_2[0], point_2[1], point_2[2])
    p3 = gp_Pnt(point_3[0], point_3[1], point_3[2])
    p4 = gp_Pnt(point_4[0], point_4[1], point_4[2])

    # 使用這些頂點創建方形的邊緣
    edge1 = BRepBuilderAPI_MakeEdge(p1, p2).Edge()
    edge2 = BRepBuilderAPI_MakeEdge(p2, p3).Edge()
    edge3 = BRepBuilderAPI_MakeEdge(p3, p4).Edge()
    edge4 = BRepBuilderAPI_MakeEdge(p4, p1).Edge()

    # 使用這些邊緣創建方形的線框
    square_wire = BRepBuilderAPI_MakeWire(edge1, edge2, edge3, edge4).Wire()

    # 將線框轉換為面，作為截面
    F = BRepBuilderAPI_MakeFace(square_wire, True)
    S=BRepOffsetAPI_MakePipe(aWire.Wire(), F.Shape())#第一个参数轨迹线,第二个参数是轮廟

    return S.Shape()

#### 等比例縮放

In [25]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.gp import gp_Trsf,gp_Pnt,gp_Vec
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Display.OCCViewer import rgb_color
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def zoom(model,m_center,mag):
    print(model,m_center,mag)
    t1 = gp_Trsf()
    #物件質心位置,單位mm
    t1.SetScale(m_center, mag)

    # 对形状进行缩放
    scaled_box = BRepBuilderAPI_Transform(model, t1, True).Shape()


    # display, start_display, add_menu, add_function_to_menu = init_display()
    # display.DisplayShape(scaled_box, update=True, color=rgb_color(0, 0, 0))
    # display.DisplayShape(model, update=True, color=rgb_color(1, 0, 0))
    # start_display()
    return scaled_box


#### 掃除

In [26]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader

def grind(model,S):
    # 创建一个几何切割对象
    cut_algo = BRepAlgoAPI_Cut(model,S)
    # 执行几何切割操作
    cut_result = cut_algo.Shape()
    return cut_result


#### 計算表面積

In [27]:
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopoDS import TopoDS_Shape

def calculate_surface_area(shape):
    # 將 shape 轉換為 TopoDS_Shape
    topods_shape = TopoDS_Shape(shape)

    # 初始化表面屬性
    surface_props = GProp_GProps()

    # 計算表面屬性
    brepgprop_SurfaceProperties(topods_shape, surface_props)
    

    # 獲取表面積
    surface_area = surface_props.Mass()

    return surface_area

#### 讀取路徑資料

In [28]:
points = []  # 创建一个空列表来存储解析后的reward值
with open(r"C:\alan\ITRI\ITRI-test\trajectory\simple1_0704_10.txt", "r") as file:
    for line in file:
        # 输出每一行内容，以确保正确读取
        print("Line:", line)
        try:
            # 从每一行中解析出8个值
            face, trajc, x, y, z, rx, ry, rz = line.split()
            # 将解析后的值作为一个元组添加到rewards列表中
            points.append((face, trajc, x, y, z, rx, ry, rz))
        except ValueError:
            # 如果发生拆包错误，输出错误信息
            print("Error: Unable to unpack values from line:", line)

# 输出解析后的rewards列表
print("points =", points)

Line: 1	1	-50.00	-50.00	-50.00	1.00	0.00	0.00

Line: 1	1	-50.00	-30.00	-50.00	1.00	0.00	0.00

Line: 1	1	-50.00	-10.00	-50.00	1.00	0.00	0.00

Line: 1	1	-50.00	10.00	-50.00	1.00	0.00	0.00

Line: 1	1	-50.00	30.00	-50.00	1.00	0.00	0.00

Line: 1	1	-50.00	50.00	-50.00	1.00	0.00	0.00

Line: 1	2	-50.00	50.00	-40.00	1.00	0.00	0.00

Line: 1	2	-50.00	30.00	-40.00	1.00	0.00	0.00

Line: 1	2	-50.00	10.00	-40.00	1.00	0.00	0.00

Line: 1	2	-50.00	-10.00	-40.00	1.00	0.00	0.00

Line: 1	2	-50.00	-30.00	-40.00	1.00	0.00	0.00

Line: 1	2	-50.00	-50.00	-40.00	1.00	0.00	0.00

Line: 1	3	-50.00	50.00	-30.00	1.00	0.00	0.00

Line: 1	3	-50.00	30.00	-30.00	1.00	0.00	0.00

Line: 1	3	-50.00	10.00	-30.00	1.00	0.00	0.00

Line: 1	3	-50.00	-10.00	-30.00	1.00	0.00	0.00

Line: 1	3	-50.00	-30.00	-30.00	1.00	0.00	0.00

Line: 1	3	-50.00	-50.00	-30.00	1.00	0.00	0.00

Line: 1	4	-50.00	50.00	-20.00	1.00	0.00	0.00

Line: 1	4	-50.00	30.00	-20.00	1.00	0.00	0.00

Line: 1	4	-50.00	10.00	-20.00	1.00	0.00	0.00

Line: 1	4	-50.00	-10.00	-

#### 組合掃除路徑

In [29]:
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
max_value_a = max(int(point[0]) for point in points)
# print("Maximum value of point[1]:", max_value_a)
max_value_b = max(int(point[1]) for point in points)

for a in range(1, max_value_a+1): 
    same_surface_points = [point for point in points if float(point[0]) ==  a]#提取同一面上的點
    # same_surface_points=[point for point in points if point[0] == '2']#提取同一面上的點
    for b in range(1, max_value_b+1): 
        same_path_points = [point for point in same_surface_points if float(point[1]) ==  b]#提取同一路徑上的點


        #same_path_points = [point for point in same_surface_points if point[1] == '1']#提取同一路徑上的點
        for i, point in enumerate(same_path_points):
            point = [float(item) for item in point]  # 将字符串转换为浮点数
            # print("Current Point:", point)
            
            point_end=np.array((point[2],point[3],point[4]))
            # print(point_end)
            normal_vector_end=np.array((point[5],point[6],point[7]))
            # print(normal_vector_end)
            if i > 0:
                previous_point = [float(item) for item in same_path_points[i - 1]]  # 获取前一个 point 的信息
                # print("Previous Point:", previous_point)
                point_start=np.array((previous_point[2],previous_point[3],previous_point[4]))
                normal_vector_start=np.array((previous_point[5],previous_point[6],previous_point[7]))
                # print(normal_vector_start)
                normal_vector=(normal_vector_end+normal_vector_start)/2
                
                # print(normal_vector)
                # print(point_start)
                # print(point_end)
                are_equal = np.array_equal(point_start, point_end)
                # if are_equal==1:
                #     point_start[0]=point_start[0]+0.001
                if are_equal==0:
                    print(point_start,point_end,normal_vector)
                    S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
                # S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
                    
                if i==1:
                    path_result=S
                        # all_result = all_result.Shape()
                else:
                    path_result = BRepAlgoAPI_Fuse(path_result,S)
                    path_result = path_result.Shape()

                # display, start_display, add_menu, add_function_to_menu = init_display()
                # box=display.DisplayShape(path_result, update=True)[0]
                # box.SetTransparency(0.8)
                # start_display()
        
        if b==1:
            surface_result=path_result
            # all_result = all_result.Shape()
        else:
            surface_result = BRepAlgoAPI_Fuse(path_result,surface_result)
            surface_result = surface_result.Shape()

    # display, start_display, add_menu, add_function_to_menu = init_display()
    # box=display.DisplayShape(surface_result, update=True)[0]
    # box.SetTransparency(0.8)
    # start_display()

    if a==1:
        all_grind_area=surface_result
        # all_result = all_result.Shape()
    else:
        all_grind_area = BRepAlgoAPI_Fuse(all_grind_area,surface_result)
        all_grind_area = all_grind_area.Shape()



display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(all_grind_area, update=True)[0]
box.SetTransparency(0.8)
start_display()

    
    # if i < len(same_path_points) - 1:
    #     next_point = [float(item) for item in same_path_points[i + 1]]  # 获取后一个 point 的信息
    #     print("Next Point:", next_point)

[-50. -50. -50.] [-50. -30. -50.] [1. 0. 0.]
[-50. -30. -50.] [-50. -10. -50.] [1. 0. 0.]
[-50. -10. -50.] [-50.  10. -50.] [1. 0. 0.]
[-50.  10. -50.] [-50.  30. -50.] [1. 0. 0.]
[-50.  30. -50.] [-50.  50. -50.] [1. 0. 0.]
[-50.  50. -50.] [-50. -50. -50.] [1. 0. 0.]
[-50. -50. -50.] [-50. -30. -50.] [1. 0. 0.]
[-50. -30. -50.] [-50. -10. -50.] [1. 0. 0.]
[-50. -10. -50.] [-50.  10. -50.] [1. 0. 0.]
[-50.  10. -50.] [-50.  30. -50.] [1. 0. 0.]
[-50.  30. -50.] [-50.  50. -50.] [1. 0. 0.]
[-50.  50. -40.] [-50.  30. -40.] [1. 0. 0.]
[-50.  30. -40.] [-50.  10. -40.] [1. 0. 0.]
[-50.  10. -40.] [-50. -10. -40.] [1. 0. 0.]
[-50. -10. -40.] [-50. -30. -40.] [1. 0. 0.]
[-50. -30. -40.] [-50. -50. -40.] [1. 0. 0.]
[-50. -50. -40.] [-50.  50. -40.] [1. 0. 0.]
[-50.  50. -40.] [-50.  30. -40.] [1. 0. 0.]
[-50.  30. -40.] [-50.  10. -40.] [1. 0. 0.]
[-50.  10. -40.] [-50. -10. -40.] [1. 0. 0.]
[-50. -10. -40.] [-50. -30. -40.] [1. 0. 0.]
[-50. -30. -40.] [-50. -50. -40.] [1. 0. 0.]
[-50.  50.

In [30]:
        
#         if b==1:
#             surface_result=path_result
#             # all_result = all_result.Shape()
#         else:
#             surface_result = BRepAlgoAPI_Fuse(path_result,surface_result)
#             surface_result = surface_result.Shape()

#     # display, start_display, add_menu, add_function_to_menu = init_display()
#     # box=display.DisplayShape(surface_result, update=True)[0]
#     # box.SetTransparency(0.8)
#     # start_display()

#     if a==1:
#         all_grind_area=surface_result
#         # all_result = all_result.Shape()
#     else:
#         all_grind_area = BRepAlgoAPI_Fuse(all_grind_area,surface_result)
#         all_grind_area = all_grind_area.Shape()



# display, start_display, add_menu, add_function_to_menu = init_display()
# box=display.DisplayShape(all_grind_area, update=True)[0]
# box.SetTransparency(0.8)
# start_display()

    
#     # if i < len(same_path_points) - 1:
#     #     next_point = [float(item) for item in same_path_points[i + 1]]  # 获取后一个 point 的信息
#     #     print("Next Point:", next_point)

In [31]:
#                 previous_point = [float(item) for item in same_path_points[i - 1]]  # 获取前一个 point 的信息
#                 # print("Previous Point:", previous_point)
#                 point_start=np.array((previous_point[2],previous_point[3],previous_point[4]))
#                 normal_vector_start=np.array((previous_point[5],previous_point[6],previous_point[7]))
#                 # print(normal_vector_start)
#                 normal_vector=(normal_vector_end+normal_vector_start)/2
                
#                 # print(normal_vector)
#                 # print(point_start)
#                 # print(point_end)
#                 are_equal = np.array_equal(point_start, point_end)
#                 # if are_equal==1:
#                 #     point_start[0]=point_start[0]+0.001
#                 if are_equal==0:
#                     print(point_start,point_end,normal_vector)
#                     S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
#                 # S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
                    
#                 if i==1:
#                     path_result=S
#                         # all_result = all_result.Shape()
#                 else:
#                     path_result = BRepAlgoAPI_Fuse(path_result,S)
#                     path_result = path_result.Shape()

#         # display, start_display, add_menu, add_function_to_menu = init_display()
#         # box=display.DisplayShape(path_result, update=True)[0]
#         # box.SetTransparency(0.8)
#         # start_display()
        
#         if b==1:
#             surface_result=path_result
#             # all_result = all_result.Shape()
#         else:
#             surface_result = BRepAlgoAPI_Fuse(path_result,surface_result)
#             surface_result = surface_result.Shape()

#     # display, start_display, add_menu, add_function_to_menu = init_display()
#     # box=display.DisplayShape(surface_result, update=True)[0]
#     # box.SetTransparency(0.8)
#     # start_display()

#     if a==1:
#         all_grind_area=surface_result
#         # all_result = all_result.Shape()
#     else:
#         all_grind_area = BRepAlgoAPI_Fuse(all_grind_area,surface_result)
#         all_grind_area = all_grind_area.Shape()



# display, start_display, add_menu, add_function_to_menu = init_display()
# box=display.DisplayShape(all_grind_area, update=True)[0]
# box.SetTransparency(0.8)
# start_display()

    
#     # if i < len(same_path_points) - 1:
#     #     next_point = [float(item) for item in same_path_points[i + 1]]  # 获取后一个 point 的信息
#     #     print("Next Point:", next_point)

In [32]:
# from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
# max_value = max(int(point[1]) for point in points)
# # print("Maximum value of point[1]:", max_value)


    
# same_surface_points = [point for point in points if float(point[0]) ==  1]#提取同一面上的點

# # same_surface_points=[point for point in points if point[0] == '2']#提取同一面上的點
# same_path_points = [point for point in same_surface_points if point[1] == '1']#提取同一路徑上的點
# for i, point in enumerate(same_path_points):
#     point = [float(item) for item in point]  # 将字符串转换为浮点数
#     # print("Current Point:", point)
        
#     point_end=np.array((point[2],point[3],point[4]))
#         # print(point_end)
#     normal_vector=np.array((point[5],point[6],point[7]))
#     if i > 0:
#         previous_point = [float(item) for item in same_path_points[i - 1]]  # 获取前一个 point 的信息
#         # print("Previous Point:", previous_point)
#         point_start=np.array((previous_point[2],previous_point[3],previous_point[4]))
#         print(point_start)
#         S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍

#         if i==1:
#             path_result=S
#                 # all_result = all_result.Shape()
#         else:
#             path_result = BRepAlgoAPI_Fuse(path_result,S)
#             path_result = path_result.Shape()

# display, start_display, add_menu, add_function_to_menu = init_display()
# box=display.DisplayShape(path_result, update=True)[0]
# box.SetTransparency(0.8)
# start_display()
    
#     # if i < len(same_path_points) - 1:
#     #     next_point = [float(item) for item in same_path_points[i + 1]]  # 获取后一个 point 的信息
#     #     print("Next Point:", next_point)

#### 對每一殼層進行掃除，並儲存掃除結果

In [33]:
from OCC.Extend.DataExchange import write_iges_file
import os
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.Extend.DataExchange import read_iges_file
from OCC.Core.Quantity import Quantity_NOC_RED, Quantity_NOC_BLUE, Quantity_NOC_GREEN


#讀取模型
iges_file =  r"C:\alan\ITRI\ITRI-test\model\simple_1.IGS"
model = read_iges_file(iges_file)
model=model[0]#如果type(model)=<class 'list'>就要加這行，正常應該是type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>

display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(model, update=True)[0]
start_display()

layer_n=6
# point_start=np.array((-40,50,0))
# point_end=np.array((40,50,0))
# normal_vector=np.array((0,1,0))
# S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
m_center= gp_Pnt(0,0,0)#質心位置
#simple_1:m_center=(0,0,0)
#simple_2:m_center=(0,-2.2,-2.2)
#simple_3:m_center=(-0.36,-0.76,-0.36)
#complex_3:m_center= gp_Pnt(0, 0, 50)
#complex_3:m_center= gp_Pnt(0, 0, 61.5)

surface_area_origin=[]
surface_area_grind=[]
colors = [Quantity_NOC_RED, Quantity_NOC_BLUE, Quantity_NOC_GREEN]  # 定义颜色列表
i=0
while i<layer_n:
    mag=1-i*0.0025
    print(model,m_center,mag)
    Model=zoom(model,m_center,mag)
    cut_result=grind(Model,all_grind_area)
    
    surface_origin = calculate_surface_area(Model)
    surface_area_origin.append(surface_origin)
    print(f"Model_{i}_origin_area:", surface_origin) 
    surface_cut_result=calculate_surface_area(cut_result)
    surface_area_grind.append(surface_cut_result)
    print(f"Model_{i}_final_area:", surface_cut_result) 


    # display, start_display, add_menu, add_function_to_menu = init_display()
    # box=display.DisplayShape(cut_result, update=True)[0]
    # box.SetTransparency(0.1)
    # start_display()



    if i==0:
        all_result=cut_result
        # all_result = all_result.Shape()
    else:
        all_result = BRepAlgoAPI_Fuse(all_result,cut_result)
        all_result = all_result.Shape()

    iges_file_path=os.path.join(r"C:\alan\ITRI\ITRI-test\cut_result",f'layer_{i}.igs' )
    write_iges_file(cut_result, iges_file_path)


    # if surface_origin== surface_cut_result:
    #     break

    i=i+1
print(surface_area_origin)
iges_file_path=os.path.join(r"C:\alan\ITRI\ITRI-test\cut_result\all.igs" )
write_iges_file(all_result, iges_file_path)



<class 'TopoDS_Compound'> <class 'gp_Pnt'> 1.0
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 1.0
Model_0_origin_area: 60000.0
Model_0_final_area: 0.0
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9975
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9975
Model_1_origin_area: 59700.375
Model_1_final_area: 59700.375


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperti

<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.995
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.995
Model_2_origin_area: 59401.5


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)


Model_2_final_area: 59401.5
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9925
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9925


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)


Model_3_origin_area: 59103.375
Model_3_final_area: 59103.375
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.99
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.99


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)


Model_4_origin_area: 58806.00000000001
Model_4_final_area: 58806.00000000001
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9875
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9875
Model_5_origin_area: 58509.37499999999
Model_5_final_area: 58509.37499999999
[60000.0, 59700.375, 59401.5, 59103.375, 58806.00000000001, 58509.37499999999]


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_9092\155294808.py:13: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(topods_shape, surface_props)


#### 檢視各層研磨結果

In [34]:
from OCC.Core.Quantity import Quantity_Color
from OCC.Extend.DataExchange import read_iges_file
from OCC.Display.SimpleGui import init_display
#讀取模型
iges_file = r"C:\alan\ITRI\ITRI-test\cut_result\layer_0.igs"
cut_result = read_iges_file(iges_file)

display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(cut_result, update=True)[0]
box.SetTransparency(0.8)
start_display()


In [35]:
#一次顯示所有殼層(彩色)
import os
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file
from OCC.Core.Quantity import Quantity_Color
from OCC.Display.OCCViewer import rgb_color

# 初始化显示窗口
display, start_display, add_menu, add_function_to_menu = init_display()

# 读取模型
iges_file_0 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_0.igs"
cut_result_0 = read_iges_file(iges_file_0)

iges_file_1 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_1.igs"
cut_result_1 = read_iges_file(iges_file_1)

iges_file_2 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_2.igs"
cut_result_2 = read_iges_file(iges_file_2)

iges_file_3 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_3.igs"
cut_result_3 = read_iges_file(iges_file_0)

iges_file_4 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_4.igs"
cut_result_4 = read_iges_file(iges_file_4)

iges_file_5 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_5.igs"
cut_result_5 = read_iges_file(iges_file_5)

# 使用 rgb_color 函数设置颜色并显示模型
display.DisplayShape(cut_result_0, update=True, color=rgb_color(0, 1, 0))  
display.DisplayShape(cut_result_1, update=True, color=rgb_color(0, 0, 1))  
# display.DisplayShape(cut_result_2, update=True, color=rgb_color(0, 0, 0.6)) 
# # display.DisplayShape(cut_result_3, update=True, color=rgb_color(0, 0, 0.4))  
# display.DisplayShape(cut_result_4, update=True, color=rgb_color(0, 0, 0.2))
# display.DisplayShape(cut_result_5, update=True, color=rgb_color(1, 0, 0))    

# 开始显示窗口
start_display()

In [36]:
#一次顯示單一殼層
from OCC.Core.Quantity import Quantity_Color
from OCC.Extend.DataExchange import read_iges_file
#讀取模型
iges_file = r"C:\alan\ITRI\ITRI-test\cut_result\layer_5.igs"
cut_result = read_iges_file(iges_file)
display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(cut_result, update=True)[0]
box.SetTransparency(0.8)
start_display()

#### 計算研磨成功率

In [37]:
success_rate=((surface_area_origin[0]-surface_area_grind[0])-(surface_area_origin[4]-surface_area_grind[4]))/surface_area_origin[0]
print(success_rate)

1.0
